<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/2.0-la-tj-ak-kc-vl-ensemble_baseline_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Sentiment Analysis in the Wild
### Ensembling Notebook | CS231n

This notebook preprocesses input videos to extract faces, frames, poses, and audio before running pre-trained models for each modality to predict group sentiment (positive, negative, or neutral). 

In [1]:

# Clone the code base
!git clone 'https://github.com/kevincong95/cs231n-emotiw.git'

# Switch to TF 1.x and navigate to the directory
%tensorflow_version 1.x
!pwd
import os
os.chdir('cs231n-emotiw')
!pwd

# Install required packages 
!pip install -r 'requirements-predictions.txt'


Cloning into 'cs231n-emotiw'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (367/367), done.
remote: Compressing objects: 100% (257/257), done.
remote: Total 621 (delta 236), reused 227 (delta 109), pack-reused 254
Receiving objects: 100% (621/621), 173.24 MiB | 34.39 MiB/s, done.
Resolving deltas: 100% (369/369), done.
TensorFlow 1.x selected.
/content
/content/cs231n-emotiw
     |████████████████████████████████| 389kB 18.4MB/s 
     |████████████████████████████████| 634kB 32.8MB/s 
     |████████████████████████████████| 3.3MB 52.7MB/s 
     |████████████████████████████████| 26.9MB 113kB/s 
     |████████████████████████████████| 337kB 56.8MB/s 
     |████████████████████████████████| 25.4MB 116kB/s 
     |████████████████████████████████| 100.2MB 31kB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-cp36-none-any.whl size=110728 sha256=cc3ad00d3aa8480fee0d6d4ede9e7f650c395db1cd7822da906b473116572631
  Stored in directory: /root/.cache/pip/wheels/e

#### Pose Pre-Requisites
Pose extraction uses the [CMU OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose) to extract body keypoints. We have pre-compiled this library for use in Colab but some system files still need to be installed. 

#### Retrieve the files

The code block below demonstrates how to retrieve the files from GCS. However, feel free to skip this step if the files are already on the local disk or you have Google Drive mounted.

In [2]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
!tar -xzf openpose.tar.gz

# The pre-built OpenPose library contains shared library files that need to be manually linked
import os
orig_lib_path = os.environ['LD_LIBRARY_PATH']
os.environ["LD_LIBRARY_PATH"] = f"{orig_lib_path}:{os.getcwd()}/openpose/build/src/openpose/:{os.getcwd()}/openpose/build/caffe/lib/"

!wget https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/val-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/test-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
!wget https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt


Selecting previously unselected package libgflags2.2.
(Reading database ... 144439 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

#### Preprocess Files

Here, we will instantiate each of the preprocessors and process all of the input video files.

NOTE: Change the input parameters as needed.

WARNING: This may take several hours to complete, depending on the number of files.

In general, pre-processing will extract the following:
- Video frames
- Pose keypoints
- Faces from each video frame
- Audio waveform and audio features

In [3]:
from src.preprocessors.preprocess_all_modes import preprocess
from src.preprocessors.pose_preprocessor import PosePreprocessor

print("Starting to preprocess train data")
preprocess(video_folder="train-tiny.zip", label_file="Train_labels.txt", local_base_path="train-tiny")

print("Starting to preprocess val data")
preprocess(video_folder="val-tiny.zip", label_file="Val_labels.txt", local_base_path="val-tiny")

print("Starting to preprocess test data")
preprocess(video_folder="test-tiny.zip", local_base_path="test-tiny")

Using TensorFlow backend.


Starting to preprocess train data
Video Preprocessor created with video_folder = train-tiny.zip , label_file = Train_labels.txt , output_folder = train-tiny-frames, output_file = train-tiny-frames.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Video Preprocessor created with video_folder = train-tiny.zip , output_folder = train-tiny-faces, output_file = train-tiny-faces.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Pose Preprocessor created with is_test = False, video_frame_folder = train-tiny-frames , output_folder = train-tiny-pose, output_file = train-tiny-pose.zip
Video Preprocessor created with video_folder = train-tiny.zip , output_folder = train-tiny-audio, output_file = train-tiny-audio.zip
Frames will be created with hop_size = 0.5
Unzipping files to temp dir train-tiny-frames_tmp...
Finished unzipping files
Found 50 videos
Processing video 1/50 with name 101_12.mp4 and class 3 

Processing video 12/50 with name 

In [0]:
# Remove the openpose folder as it is no longer required
!rm -rf openpose/

In [5]:
!ls

data			      test-tiny-frames	     train-tiny-pose
LICENSE			      test-tiny-frames_tmp   train-tiny-pose.zip
models			      test-tiny-frames.zip   train-tiny.zip
notebooks		      test-tiny-pose	     Val_labels.txt
openpose.tar.gz		      test-tiny-pose.zip     val-tiny-audio
README.md		      test-tiny.zip	     val-tiny-audio_tmp
reports			      Train_labels.txt	     val-tiny-audio.zip
requirements-predictions.txt  train-tiny-audio	     val-tiny-faces
requirements.txt	      train-tiny-audio_tmp   val-tiny-faces_tmp
src			      train-tiny-audio.zip   val-tiny-faces.zip
test-tiny-audio		      train-tiny-faces	     val-tiny-frames
test-tiny-audio_tmp	      train-tiny-faces_tmp   val-tiny-frames_tmp
test-tiny-audio.zip	      train-tiny-faces.zip   val-tiny-frames.zip
test-tiny-faces		      train-tiny-frames      val-tiny-pose
test-tiny-faces_tmp	      train-tiny-frames_tmp  val-tiny-pose.zip
test-tiny-faces.zip	      train-tiny-frames.zip  val-tiny.zip


In [6]:
!ls

data			      test-tiny-frames	     train-tiny-pose
LICENSE			      test-tiny-frames_tmp   train-tiny-pose.zip
models			      test-tiny-frames.zip   train-tiny.zip
notebooks		      test-tiny-pose	     Val_labels.txt
openpose.tar.gz		      test-tiny-pose.zip     val-tiny-audio
README.md		      test-tiny.zip	     val-tiny-audio_tmp
reports			      Train_labels.txt	     val-tiny-audio.zip
requirements-predictions.txt  train-tiny-audio	     val-tiny-faces
requirements.txt	      train-tiny-audio_tmp   val-tiny-faces_tmp
src			      train-tiny-audio.zip   val-tiny-faces.zip
test-tiny-audio		      train-tiny-faces	     val-tiny-frames
test-tiny-audio_tmp	      train-tiny-faces_tmp   val-tiny-frames_tmp
test-tiny-audio.zip	      train-tiny-faces.zip   val-tiny-frames.zip
test-tiny-faces		      train-tiny-frames      val-tiny-pose
test-tiny-faces_tmp	      train-tiny-frames_tmp  val-tiny-pose.zip
test-tiny-faces.zip	      train-tiny-frames.zip  val-tiny.zip


In [7]:
!cp train-tiny-*.zip "/content/drive/My Drive/"
!cp val-tiny-*.zip "/content/drive/My Drive/"
!cp test-tiny-*.zip "/content/drive/My Drive/"

cp: target '/content/drive/My Drive/' is not a directory
cp: target '/content/drive/My Drive/' is not a directory
cp: target '/content/drive/My Drive/' is not a directory


### Run Classifiers

**IMPORTANT**: You must restart the runtime at this point to use TF 2.x

In [0]:
%tensorflow_version 2.x

In [2]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [4]:
!pwd
import os
os.chdir('cs231n-emotiw')
!pwd


/content
/content/cs231n-emotiw


In [5]:
!git pull

Already up to date.


In [6]:
from src.classifiers.audio_classifier import AudioClassifier
from src.classifiers.frames_classifier import FramesClassifier
from src.classifiers.pose_classifier import PoseClassifier
from src.classifiers.utils import get_num_samples
import numpy as np

audio_classifier = AudioClassifier('train-tiny-audio', model_location='https://storage.googleapis.com/cs231n-emotiw/models/openl3-cnn-lstm-tuned-lr.h5', is_test=False)
frames_classifier = FramesClassifier('train-tiny-frames', model_location='https://storage.googleapis.com/cs231n-emotiw/models/frame-classifier-resnet-lstm-x3.h5', is_test=False)
pose_classifier = PoseClassifier('train-tiny-pose', model_location='https://storage.googleapis.com/cs231n-emotiw/models/pose-classifier-v5.h5', is_test=False)

classifiers = [audio_classifier, frames_classifier, pose_classifier]

sample_to_true_label = {}
with open("Train_labels.txt") as f:
    l = 0
    for line in f:
        if l == 0:
            # Skip headers
            l += 1
            continue
        line_arr = line.split(" ")
        sample_to_true_label[line_arr[0].strip()] = int(line_arr[1].strip())
        l += 1



AudioClassifier created with audio_folder = train-tiny-audio , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/openl3-cnn-lstm-tuned-lr.h5
17227776/17223968 [==============================] - 0s 0us/step
FramesClassifier created with frames_folder = train-tiny-frames , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/frame-classifier-resnet-lstm-x3.h5
202964992/202962720 [==============================] - 4s 0us/step
PoseClassifier created with pose_folder = train-tiny-pose , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/pose-classifier-v5.h5
81920/81320 [==============================] - 0s 0us/step


In [8]:
audio_classifier.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 6144)]      0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          786496    
_________________________________________________________________
batch_normalization (BatchNo (None, None, 64)          256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 512)         66048     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         2048  

In [9]:
frames_classifier.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 12, 320, 480 0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 12, 10, 15, 2 23587712    input_4[0][0]                    
__________________________________________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)     (None, 12, 10, 15, 4 3006880     time_distributed_1[0][0]         
__________________________________________________________________________________________________
conv_lst_m2d_4 (ConvLSTM2D)     (None, 12, 10, 15, 4 3006880     time_distributed_1[0][0]         
____________________________________________________________________________________________

In [10]:
pose_classifier.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12, 27)]          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                15360     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 15,555
Trainable params: 15,555
Non-trainable params: 0
_________________________________________________________________


In [0]:
def predict(classifiers, sample_to_true_label=None, mode="soft", complex_fusion=False):
    assert mode in ["soft" , "hard"]

    classifier_outputs = []
    classifier_samples = []
    sample_to_row = {}
    num_samples = 0
    y_true = []

    for c, classifier in enumerate(classifiers):
        results, samples = classifier.predict()
        classifier_outputs.append(results.tolist())
        classifier_samples.append(list(samples))
        num_samples = len(list(samples))

    print(f"Number of samples: {num_samples}")

    for i, sample in enumerate(classifier_samples[0]):
        sample_to_row[sample] = i
        if sample_to_true_label is not None:
            y_true.append(sample_to_true_label[sample])

    X = np.zeros(shape=(len(classifiers), num_samples, 3))
    for c, output in enumerate(classifier_outputs):
        samples = classifier_samples[c]
        for i, row in enumerate(output):
            sample = samples[i]
            X[c, sample_to_row[sample], :] += row


    if mode == "soft":
        # Take the average of each 
        y_pred = np.mean(X, axis=0)
        y_pred = np.argmax(y_pred, axis=1) + 1 # Add 1 because true labels range from 1 to 3
        if sample_to_true_label is not None:
            y_true = np.array(y_true)

        print("Predicted y-labels:")
        print(y_pred)

        if sample_to_true_label is not None:
            print("True y-labels:")
            print(y_true)

            accuracy = (y_pred == y_true).mean()
            print(f"Accuracy: {accuracy}")
    else:
        print("Not implemented yet")


In [12]:
from src.classifiers.audio_classifier import AudioClassifier
from src.classifiers.frames_classifier import FramesClassifier
from src.classifiers.pose_classifier import PoseClassifier
from src.classifiers.utils import get_num_samples
import numpy as np

audio_classifier = AudioClassifier('val-tiny-audio', model_location='https://storage.googleapis.com/cs231n-emotiw/models/OPENL3_audio_api_train_test-1-500-epochs-0.5_hop--BEST_MODEL-w-VAL--1e-6-lr-0.2-dropout-512-feat-map-batch-norm-3-cnn-layers.h5', is_test=False)
frames_classifier = FramesClassifier('val-tiny-frames', model_location='https://storage.googleapis.com/cs231n-emotiw/models/frame-classifier-resnet-lstm-x3.h5', is_test=False)
pose_classifier = PoseClassifier('val-tiny-pose', model_location='https://storage.googleapis.com/cs231n-emotiw/models/pose-classifier-v5.h5', is_test=False)

# classifiers = [audio_classifier, frames_classifier, pose_classifier]
classifiers = [frames_classifier]

sample_to_true_label = {}
with open("Val_labels.txt") as f:
    l = 0
    for line in f:
        if l == 0:
            # Skip headers
            l += 1
            continue
        line_arr = line.split(" ")
        sample_to_true_label[line_arr[0].strip()] = int(line_arr[1].strip())
        l += 1
print(sample_to_true_label)


AudioClassifier created with audio_folder = val-tiny-audio , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/OPENL3_audio_api_train_test-1-500-epochs-0.5_hop--BEST_MODEL-w-VAL--1e-6-lr-0.2-dropout-512-feat-map-batch-norm-3-cnn-layers.h5
FramesClassifier created with frames_folder = val-tiny-frames , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/frame-classifier-resnet-lstm-x3.h5
PoseClassifier created with pose_folder = val-tiny-pose , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/pose-classifier-v5.h5
{'1_1': 3, '1_2': 3, '5_1': 1, '5_2': 1, '5_3': 1, '5_4': 1, '5_5': 1, '5_6': 1, '5_7': 1, '5_8': 1, '5_9': 1, '5_10': 1, '5_11': 1, '5_12': 1, '5_13': 1, '5_14': 1, '5_15': 1, '5_16': 1, '5_17': 1, '5_18': 1, '5_19': 1, '11_1': 2, '11_2': 1, '11_3': 1, '11_4': 1, '11_5': 1, '11_6': 1, '11_7': 1, '11_8': 1, '11_9': 1, '11_10': 1, '11_11': 1, '11_12': 1, '11_13': 2, '11

In [14]:
predict(classifiers, sample_to_true_label)

Skipping unzipping files as input is a folder
Found 50 frames belonging to 50 videos belonging to 3 classes.
Min frames determined to be 13
Number of samples: 50
Predicted y-labels:
[2 2 2 1 1 2 1 2 3 1 2 2 1 3 2 3 2 1 3 1 1 1 3 3 3 3 1 3 1 2 3 1 2 2 2 2 2
 2 2 2 2 2 1 2 2 3 1 1 1 1]
True y-labels:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
Accuracy: 0.54
